## SimpleGeo.py
<hr style="border:2px solid #0077b9;">

- Dados de Queimadas
- SimpleGeo.py

# Dados de Queimadas
<hr style="border:2px solid #0077b9;">

<table style="font-size: 26px;">
<thead>
<tr>
<th style="text-align: left">#</th>
<th style="text-align: left">Dado</th>
<th>Descrição</th>
<th style="text-align: center">Formato</th>
<th style="text-align: center">Res. Temp.</th>
<th style="text-align: center">Res. Espacial</th>
<th style="text-align: center">Dimensões</th>
<th style="text-align: center">Tipo</th>
<th style="text-align: center">Serviços</th>
<th style="text-align: center">Tamanho</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align: left">1</td>
<td style="text-align: left">Focos de Queimadas</td>
<td>49.755.288 registros</td>
<td style="text-align: center">PostgreSQL</td>
<td style="text-align: center">Dados obtidos em 2017-08-11</td>
<td style="text-align: center">-</td>
<td style="text-align: center">-</td>
<td style="text-align: center">Feature</td>
<td style="text-align: center">WFS</td>
<td style="text-align: center">5GB+10GB</td>
</tr>
<tr>
<td style="text-align: left">2</td>
<td style="text-align: left">Risco de Fogo</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
<tr>
<td style="text-align: left">3</td>
<td style="text-align: left">Precipitação</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
<tr>
<td style="text-align: left">4</td>
<td style="text-align: left">Temperatura</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
<tr>
<td style="text-align: left">5</td>
<td style="text-align: left">Humidade</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
</tbody>
</table>

<table>
<tr>
<td><img src="./imgs/focos_bra_2016.png" alt="Focos BRA 2016" style="height: 250px;"/></td>
<td><img src="./imgs/risco.png" alt="Risco 2016" style="height: 250px;"/></td>
<td><img src="./imgs/precipitacao.png" alt="Precipitação 2016" style="height: 250px;"/></td>
<td><img src="./imgs/temperatura.png" alt="Temperatura 2016" style="height: 250px;"/></td>
</tr>
<tr>
<td style="text-align: center">1. Focos BRA 2016</td>
<td style="text-align: center">2. Risco 01-01-2016</td>
<td style="text-align: center">3. Precipitação 01-01-2016</td>
<td style="text-align: center">4. Temperatura 01-01-2016</td>
</tr>
<tr>
<td><img src="./imgs/umidade.png" alt="Umidade 2016" style="height: 250px;"/></td>
</tr>
<tr>
<td style="text-align: center">5. Umidade 01-01-2016</td>
</tr>
</table>

# SimpleGeo.py
<hr style="border:2px solid #0077b9;">

A SimpleGeo.py é uma biblioteca para o acesso e integração de dados geospaciais através de serviços WFS e WTSS.

## Repositório GIT
<hr style="border:2px solid #0077b9;">
<img src="./imgs/simplegeo.git.png" style="width: 1200px;"/>
URL: [github.com/e-sensing/simple_geo.py](github.com/e-sensing/simple_geo.py)

## Usando a SimpleGeo.py
<hr style="border:2px solid #0077b9;">

### Listando Features e Coverages disponíveis
<hr style="border:2px solid #0077b9;">

In [1]:
from SimpleGeo import SimpleGeo

# connect to wfs and wtss servers
s = SimpleGeo(wfs="http://www.terrama2.dpi.inpe.br/geoserver", wtss="http://www.terrama2.dpi.inpe.br/e-sensing")

# print available features
print("Features:")
for f in s.features()['features']:
    print("\t{}".format(f))

# print available coverages
print("\nCoverages:")
for c in s.coverages()['coverages']:
    print("\t{}".format(c))

Features:
	esensing:estados_bra
	esensing:focos_bra_2016
	esensing:municipios_bra

Coverages:
	rpth


## Obtendo Metadados das Features e Coverages
<hr style="border:2px solid #0077b9;">

In [ ]:
# Feature estados_bra
f = s.feature('esensing:estados_bra')

metadata = f.describe()

#print(metadata)

print("Nome: {}".format(metadata['full_name']))
print("Tipo: {}".format(metadata['geometry']['localtype']))

print("Atributos:")
for att in metadata['attributes']:
    print("\t{:<12}: {}".format(att['name'], att['localtype']))

In [ ]:
# Coverage rpth
c = s.coverage('rpth')

metadata = c.describe()
print("Nome: {}".format(metadata['name']))
print("Proj4: {}".format(metadata['crs']['proj4']))
print("Timeline: {} ... {}".format(metadata['timeline'][1], metadata['timeline'][-1]))
print("Atributos:")
for att in metadata['attributes']:
    print("\t{}".format(att))

#print(metadata)

## Obtendo dados de Features
<hr style="border:2px solid #0077b9;">

In [ ]:
f = s.feature('esensing:estados_bra')
estados = f.get()
# displaying 
display(estados.head(n=10))

## Métodos da SimpleGeo para Features
<hr style="border:2px solid #0077b9;">

### Métodos para Features

- **attributes**: define quais atributos devem ser retornados
- **max_features**: limita o número de features a serem retornadas
- **sort_by**: ordena os elementos por um atributo não espacial
- **filter**: seleciona por um ou mais atributos (espaciais ou não)



### Selecionando atributos de Features (attributes)
<hr style="border:2px solid #0077b9;">

In [ ]:
f = s.feature('esensing:estados_bra').attributes(['nome','regiao'])
estados = f.get()
# displaying 
display(estados.head(n=10))

### Limitando o número de retornos (max_features)
<hr style="border:2px solid #0077b9;">

In [ ]:
f = s.feature('esensing:estados_bra').attributes(['nome','regiao']).max_features(5)
estados = f.get()
# displaying
display(estados)

### Ordenando features (sort_by)
<hr style="border:2px solid #0077b9;">

In [ ]:
# Ascendente ASC
f = s.feature('esensing:estados_bra').attributes(['nome','regiao']).max_features(5).sort_by('nome')
estados = f.get()
# displaying
display(estados)

In [ ]:
from SimpleGeo import Predicates as pre

# Descesdente DESC
f = s.feature('esensing:estados_bra').attributes(['nome','regiao']).max_features(5).sort_by(pre.DESC('nome'))
estados = f.get()
# displaying
display(estados)

### Filtrando Features
<hr style="border:2px solid #0077b9;">

A **SimpleGeo** fornece os seguintes predicados para auxiliar a construção de um filtro:

- Atributos não espaciais:
    - EQ: igual
    - NE: diferente
    - LT: menor que
    - GT: maior que
    - LE: menor ou igual a
    - GE: maior ou igual a
    - AND: e
    - OR: ou

- Atributos espaciais:
    - WITHIN: dentro
    - INTERSECTS: intersecta

In [ ]:
# Only South region
f = s.feature('esensing:estados_bra')\
     .filter(pre.EQ('regiao','S'))
estados = f.get()
# displaying 
display(estados)

In [ ]:
from SimpleGeo import Predicates as pre

# South or North regions
f = s.feature('esensing:estados_bra')\
     .filter(pre.OR(
             pre.EQ('regiao','S'),
             pre.EQ('regiao','N')
         ))
estados = f.get()
# displaying 
display(estados)

In [ ]:
from SimpleGeo import Predicates as pre

f = s.feature("esensing:focos_bra_2016") \
    .attributes(["id", "municipio", "timestamp", "regiao"]) \
    .filter(pre.OR(
    pre.AND(
        pre.EQ("satelite_referencia", "true"),
        pre.GE("timestamp", "2016-01-01"),
        pre.LT("timestamp", "2016-02-01"),
        pre.WITHIN(
            "POLYGON((-49.515628859948507 -19.394602563415745,-48.020567850467053 -19.610579617637825,-48.354439522883652 -21.052347219666608,-49.849500507163917 -20.836369963642884,-49.515628859948507 -19.394602563415745))"),
    ), \
    pre.NE("regiao", "SE") \
    )) \
    .max_features(10) \
    .sort_by(pre.DESC("municipio"))
    
focos = f.get()
display(focos)

## Obtendo Séries Temporais com a SimpleGeo
<hr style="border:2px solid #0077b9;">

In [13]:
from shapely.geometry import Point
# Time Series

c = s.coverage("rpth") \
    .attributes(["precipitation", "risk", "temperature", "humidity"])

ts = s.time_serie(c) \
    .period("2016-01-01", "2016-01-04")

ts_data = ts.get(Point(-54.0, -12.0))
display(ts_data)

,humidity,precipitation,risk,temperature
2016-01-01,0.620963,0.840492,0.047310,303.245178
2016-01-02,0.531946,5.364658,0.021442,303.892456
2016-01-03,0.734242,2.840686,0.029456,300.913147
2016-01-04,0.535269,3.866830,0.034228,304.197449


In [14]:
# Lista de Pontos
ts_data = ts.get([Point(-54.0, -12.0), Point(-54.0, -13.0)])
for t in ts_data:
    display(t)

,humidity,precipitation,risk,temperature
2016-01-01,0.620963,0.840492,0.047310,303.245178
2016-01-02,0.531946,5.364658,0.021442,303.892456
2016-01-03,0.734242,2.840686,0.029456,300.913147
2016-01-04,0.535269,3.866830,0.034228,304.197449


,humidity,precipitation,risk,temperature
2016-01-01,0.570955,1.923350,0.042945,303.669464
2016-01-02,0.667068,5.284581,0.024789,301.866028
2016-01-03,0.790824,5.740402,0.018523,299.142426
2016-01-04,0.460336,0.241759,0.102072,305.305603


## Integrando Features e Séries Temporais
<hr style="border:2px solid #0077b9;">

In [18]:
# Integrando Features e TimeSeries

# Série Temporal
c = s.coverage("rpth") \
    .attributes(["precipitation", "risk", "temperature", "humidity"])
ts = s.time_serie(c)

# Feature
f = s.feature("esensing:focos_bra_2016") \
    .attributes(["id", "municipio", "timestamp", "regiao"]) \
    .sort_by("timestamp") \
    .max_features(5)
display(f.get())

,bbox,geometry,id,municipio,regiao,timestamp
0,"[-52.31, -2.95, -52.31, -2.95]",POINT (-52.31 -2.95),45276709,Vitória Do Xingu,N,2016-01-01T00:00:00Z
1,"[-63.68, 0.5, -63.68, 0.5]",POINT (-63.68 0.5),25231226,Barcelos,N,2016-01-01T00:00:00Z
2,"[-63.66, 0.5, -63.66, 0.5]",POINT (-63.66 0.5),25239660,Barcelos,N,2016-01-01T00:00:00Z
3,"[-52.33, -2.95, -52.33, -2.95]",POINT (-52.33 -2.95),42899497,Brasil Novo,N,2016-01-01T00:00:00Z
4,"[-61.1, -11.34, -61.1, -11.34]",POINT (-61.1 -11.34),46941054,Cacoal,N,2016-01-01T00:00:00Z


In [16]:
# Integrando Features e TimeSeries

# Série Temporal
c = s.coverage("rpth") \
    .attributes(["precipitation", "risk", "temperature", "humidity"])
ts = s.time_serie(c)

# Feature + Série Temporal
f = s.feature("esensing:focos_bra_2016") \
    .attributes(
    ["id", "municipio", "timestamp", "regiao",
     {'time_serie': ts, 'start_date': 0, 'end_date': 0, 'datetime': 'timestamp'}]) \
    .sort_by("timestamp") \
    .max_features(5)
display(f.get())

,bbox,geometry,id,municipio,regiao,timestamp,humidity,precipitation,risk,temperature
0,"[-52.31, -2.95, -52.31, -2.95]",POINT (-52.31 -2.95),45276709,Vitória Do Xingu,N,2016-01-01T00:00:00Z,0.565847,2.938754,0.536405,304.520020
1,"[-63.68, 0.5, -63.68, 0.5]",POINT (-63.68 0.5),25231226,Barcelos,N,2016-01-01T00:00:00Z,0.473713,0.000000,0.415944,305.005859
2,"[-63.66, 0.5, -63.66, 0.5]",POINT (-63.66 0.5),25239660,Barcelos,N,2016-01-01T00:00:00Z,0.473829,0.000000,0.438030,305.018280
3,"[-52.33, -2.95, -52.33, -2.95]",POINT (-52.33 -2.95),42899497,Brasil Novo,N,2016-01-01T00:00:00Z,0.567601,2.650581,1.000000,304.486359
4,"[-61.1, -11.34, -61.1, -11.34]",POINT (-61.1 -11.34),46941054,Cacoal,N,2016-01-01T00:00:00Z,0.623452,0.027066,0.154713,302.489166


### Cache
<hr style="border:2px solid #0077b9;">


In [20]:
from SimpleGeo import Predicates as pre
import time

# connect to wfs and wtss servers
s = SimpleGeo(wfs="http://www.terrama2.dpi.inpe.br/geoserver", wtss="http://www.terrama2.dpi.inpe.br/e-sensing", cache=True, cache_dir="/tmp/.cache")
s.clear_cache()
f = s.feature("esensing:focos_bra_2016") \
    .attributes(["id", "municipio", "timestamp", "regiao"]) \
    .filter(pre.OR(
    pre.AND(
        pre.EQ("satelite_referencia", "true"),
        pre.GE("timestamp", "2016-01-01"),
        pre.LT("timestamp", "2016-02-01"),
        pre.WITHIN(
            "POLYGON((-49.515628859948507 -19.394602563415745,-48.020567850467053 -19.610579617637825,-48.354439522883652 -21.052347219666608,-49.849500507163917 -20.836369963642884,-49.515628859948507 -19.394602563415745))"),
    ), \
    pre.NE("regiao", "SE") \
    )) \
    .max_features(10) \
    .sort_by(pre.DESC("municipio"))

start = time.time()
focos = f.get()
end = time.time()

# total time
print("Elapsed time: {}s".format(end-start))

Elapsed time: 9.74682998657s
